In [1]:

import gradio as gr
import time
from tqdm import tqdm
import re
import modules.callbacks
import PIL
import base64
import io
import requests

In [2]:
def count_sentences(text):
    #From text, count how many . ! ? there are
    #If . ! ? are followed by a space,double mark or a new line, then it is a sentence
    #If . ! ? in the end of the text, then it is a sentence
    return len(re.findall(r'[\.\!\?][\s\”\'\n]',text)) + len(re.findall(r'[\.\!\?]$',text))


### Translation Model (Help Txt2Img)

In [3]:
from transformers import pipeline

id_to_en_pipeline = pipeline("translation_id_to_en", model="Helsinki-NLP/opus-mt-id-en", tokenizer="Helsinki-NLP/opus-mt-id-en", device='cpu')

2023-03-26 14:50:54.773112: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 14:50:56.429590: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/lib/wsl/lib::/home/karuniaperjuangan/anaconda3/lib::/usr/local/cuda-11.2/targets/x86_64-linux/lib::
2023-03-26 14:50:56.431363: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/lib/wsl/lib::/home

### GPT Model

In [4]:
import gc
import torch

def clear_torch_cache():
    gc.collect()
    torch.cuda.empty_cache()

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path_or_name = "/home/karuniaperjuangan/AI-Project/text-generation-webui/models/gpt2-medium-indonesian-story"

model = AutoModelForCausalLM.from_pretrained(model_path_or_name,
                                             load_in_8bit=True,
                                             device_map="auto"
                                             ).cuda()
tokenizer = AutoTokenizer.from_pretrained(model_path_or_name,truncation_side="left")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /home/karuniaperjuangan/anaconda3/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/karuniaperjuangan/anaconda3/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


In [6]:
def stream_generate_text(init_text,max_sentences=0,max_tokens=512,hidden_texts=[]):
    """
    init_text: initial text that will be used to generate text
    max_sentences: maximum number of sentences to generate
    max_tokens: maximum number of tokens to generate
    hidden_sentence: list of string that will be hidden from the generated text (example: initial text and unfinished sentences)
    """
    sentence = init_text
    inputs_ids = tokenizer(sentence, return_tensors="pt").input_ids.cuda()
    continue_generate = True
    while continue_generate:
        inputs_ids = tokenizer(sentence, return_tensors="pt").input_ids.cuda()
        generate_params = {
        "max_new_tokens": 8,
        "eos_token_id": tokenizer.eos_token_id,
        "pad_token_id": tokenizer.eos_token_id,
        "top_k": 50,
        "top_p": 0.95,
        "temperature": 0.9,
        "repetition_penalty": 1.2,
        "do_sample": True,
        }
        generate_params["input_ids"] = inputs_ids
        outputs = model.generate(**generate_params)
        sentence = tokenizer.decode(outputs[0])
        inputs_ids = outputs[0]
        #print(count_sentences(sentence)) 
        outputted_sentences = sentence
        for hidden in hidden_texts:
            outputted_sentences = outputted_sentences.replace(hidden,"")
        yield outputted_sentences
            
        if count_sentences(sentence) >= max_sentences or ( len(inputs_ids)-len(tokenizer.encode(init_text)) ) > max_tokens or tokenizer.eos_token_id in inputs_ids:
            continue_generate = False
    #Clear text after last ,.",!",?",.,?,!
    deleted_text = re.split(r"[\.\!\?]",sentence)[-1]

    outputted_sentences = sentence.replace(deleted_text,"")
    for hidden in hidden_texts:
        outputted_sentences = outputted_sentences.replace(hidden,"")
    yield outputted_sentences

In [7]:
story_sequence = []

def generate_story_and_choices(beginning_text, selected_choice="", num_choices=3, story_beginning=True):
    clear_torch_cache()
    """
    beginning_text: text that will be used to generate story (if in beginning is the story, else the choice of previous story)
    num_choices: number of choices to generate
    story_beginning: True if the beginning_text is the story, else False
    """
    global story_sequence
    if story_beginning:
        story_sequence = []
        story = beginning_text
    else:
        story = ""
        for item in story_sequence:
            story += item
    pregenerated_length = len(story)
    
    story += selected_choice

    
    choices = ["..." for i in range(num_choices)]

    for text in stream_generate_text(story,max_sentences=5,max_tokens=512):
        story = text
        # Di awal, juga tampilkan awalan cerita yang dimasukkan pengguna
        yield story[pregenerated_length:] if not story_beginning else story, *choices
    
    #clean story from html tag
    story = re.sub(r"<[^>]*>","",story)
    
    story_sequence.append(story[pregenerated_length:] if not story_beginning else story)
    #Generate choices
    max_sentences = 2+count_sentences(story)
    for i in range(num_choices):
        choices[i] = ""
        for text in stream_generate_text(story,max_sentences=max_sentences,max_tokens=512):
            choices[i] = text[len(story):]
            yield story[pregenerated_length:] if not story_beginning else story, *choices
        #clean choices from html tag
        choices[i] = re.sub(r"<[^>]*>","",choices[i])
        
def select_choice(choice):
    global story_sequence
    full_story = ""
    for item in story_sequence:
        full_story += item
    for iter in generate_story_and_choices("",selected_choice=choice,story_beginning=False):
        yield iter

In [8]:
from PIL import Image
blank_image = Image.new('RGB', (768, 768), (255,255,255))

### Stable Diffusion API

In [9]:
url_api = "http://127.0.0.1:3760/sdapi/v1/txt2img"

In [10]:
def create_illustration(story="",magic_prompt="",negative_prompt="((nsfw)),ugly,deformed,jpeg artifacts, signature, watermark, username, artist name"):
    """
    story: story to be illustrated
    magic_prompt: magic prompt to be used to generate illustration
    """
    try:
        response = requests.post(url_api, json={
        "prompt": f'"{story}", {magic_prompt} ',
        "seed": -1,
        "steps": 20,
        "cfg_scale": 7,
        "width": 768,
        "height": 768,
        "negative_prompt": negative_prompt
        }).json()
        image = Image.open(io.BytesIO(base64.b64decode(response["images"][0])))
        return image
    except Exception as e:
        print("Error generating illustration:",e)
        return blank_image


In [11]:
def illustration_from_story(story,magic_prompt="illustration, kid story, samdoesart style"):
    #Take the first 2 sentences of the story
    story_sentences = re.split(r"[\.\!\?]",story)
    story_sentences = "".join(story_sentences[:2])
    #Translate prompt from Indonesian to English
    prompt = id_to_en_pipeline(story_sentences)[0]["translation_text"]
    #clean prompt from html tag
    prompt = re.sub(r"<[^>]*>","",prompt)
    #Generate illustration
    image = create_illustration(story=prompt,magic_prompt=magic_prompt)
    return [image]

### Gradio Demo

In [12]:
with gr.Blocks(css='./style.css') as blocks:
    with gr.Column():
        with gr.Row():
            text = gr.Textbox(label="Text", value="Pada zaman dahulu,")
            button = gr.Button(value="Generate")
        with gr.Column():
            with gr.Row():
                output = gr.Markdown(value="This is a test")
                #image = gr.Gallery(value=[blank_image])
            with gr.Row():
                with gr.Column():
                    with gr.Row(elem_classes="choicerow"):
                        button_1 = gr.Button(value="Select", elem_classes="choicebutton")
                        choice_1 = gr.Markdown(value="This is a choice 1",elem_classes="choice-label")
                    with gr.Row(elem_classes="choicerow"):
                        button_2 = gr.Button(value="Select", elem_classes="choicebutton")
                        choice_2 = gr.Markdown(value="This is a choice 1",elem_classes="choice-label")
                    with gr.Row(elem_classes="choicerow"):
                        button_3 = gr.Button(value="Select", elem_classes="choicebutton")
                        choice_3 = gr.Markdown(value="This is a choice 1",elem_classes="choice-label")
                images = gr.Gallery(value=[blank_image])    
            with gr.Row():
                magic_prompt = gr.Textbox(label="Magic Prompt", value="Detail, Sharp focus, dramatic, RAW photo, 8k uhd, film grain")
                
                

        with gr.Row():
            pass
    button.click(generate_story_and_choices,[text],[output,choice_1,choice_2,choice_3]).then(illustration_from_story,inputs=[output,magic_prompt],outputs=[images])
    button_1.click(select_choice,[choice_1],[output,choice_1,choice_2,choice_3]).then(illustration_from_story,inputs=[output,magic_prompt],outputs=[images])
    button_2.click(select_choice,[choice_2],[output,choice_1,choice_2,choice_3]).then(illustration_from_story,inputs=[output,magic_prompt],outputs=[images])
    button_3.click(select_choice,[choice_3],[output,choice_1,choice_2,choice_3]).then(illustration_from_story,inputs=[output,magic_prompt],outputs=[images])
        
        
blocks.queue().launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/home/karuniaperjuangan/anaconda3/lib/python3.9/site-packages/transformers/models/gpt2/modeling_gpt2.py:201: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/TensorCompare.cpp:413.)
  attn_weights = torch.where(causal_mask, attn_weights.to(attn_weights.dtype), mask_value)
